<a href="https://colab.research.google.com/github/farhanhubble/google-colab/blob/master/advanced_lane_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [0]:
!ls /content/drive/My\ Drive/data_driver_view | head -10

In [0]:
def plot_img_grid(images, nb_cols):
    nb_images = len(images)
    nb_rows = int(np.ceil(nb_images/nb_cols))
    
    for i in range(nb_rows):
        plt.figure(figsize=(32, 4))
        for j in range(nb_cols):
            plt.subplot(1, nb_cols, j+1)
            image = images[i*nb_cols + j]
            nb_channels = 1 if len(image.shape) == 2 else image.shape[-1]
            plt.imshow(image, cmap='gray' if nb_channels == 1 else None)
            plt.xticks([])
            plt.yticks([])
        plt.show()

In [0]:
import os
from glob import glob
from tqdm import tqdm

if os.path.exists('./cache/road_data.npy'):
  print('Loading data from cache...')
  road_data = np.load('./cache/road_data.npy')
  print(f'Data with dimensions {road_data.shape} loaded.')
else:
  road_data = []
  os.makedirs('./cache',exist_ok=True)
  filenames = glob('/content/drive/My Drive/data_driver_view/*.png')
  print('Loading data from Google Drive')
  for name in tqdm(filenames):
    img_bgr = cv2.imread(name)
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    road_data.append(img_rgb)
  np.save('./cache/road_data.npy',road_data)
  print(f'{len(road_data)} images loaded')

In [0]:
random_indices = np.random.choice(range(len(road_data)),64)
plot_img_grid([road_data[i] for i in random_indices ],4)

In [0]:
grad_x = cv2.Sobel(road_data[0], cv2.CV_16S, 1, 0,)
grad_y = cv2.Sobel(road_data[0], cv2.CV_16S, 0, 1)

grad_x_abs = np.abs(grad_x)
grad_y_abs = np.abs(grad_y)

# Scaled gradients 
max_channelwise_x = np.max(grad_x_abs, axis=(0,1))
min_channelwise_x = np.min(grad_x_abs, axis=(0,1))

max_channelwise_y = np.max(grad_y_abs, axis=(0,1))
min_channelwise_y = np.min(grad_y_abs, axis=(0,1))

grad_x_scaled = ((grad_x_abs - min_channelwise_x)/(max_channelwise_x - min_channelwise_x))
grad_y_scaled = ((grad_y_abs - min_channelwise_y)/(max_channelwise_y - min_channelwise_y))

In [0]:
plt.matshow(grad_x_scaled); plt.show();
plt.matshow(grad_y_scaled); plt.show();

In [0]:
# Automatic thresholding
bin_grad_x = cv2.threshold((255*grad_x_scaled[:,:,0]).astype(np.uint8), 0, 255, cv2.THRESH_OTSU+cv2.THRESH_BINARY)
bin_grad_y = cv2.threshold((255*grad_y_scaled[:,:,0]).astype(np.uint8), 0, 255, cv2.THRESH_OTSU+cv2.THRESH_BINARY)

In [0]:
plt.matshow(bin_grad_x[1], cmap='gray')

In [0]:
plt.matshow(bin_grad_y[1], cmap='gray')

In [0]:
grad_combined = np.sqrt( np.square(grad_x_scaled) + np.square(grad_y_scaled)) 

# Find the connected compnents

# Scale the gradients
max_combined = np.max(grad_combined, axis=(0,1))
min_combined = np.min(grad_combined, axis=(0,1))


scaled_grad_combined = ((grad_combined - min_combined)/(max_combined - min_combined))

# Convert to binary value
bin_grad_combined = cv2.threshold((255*scaled_grad_combined[:,:,0]).astype(np.uint8), 
                                  0, 255, cv2.THRESH_OTSU+cv2.THRESH_BINARY)

#find the connected components
retval, labels, stats, centroids = cv2.connectedComponentsWithStats(bin_grad_combined[1])

# Find the top 10 components
top_components_index = np.argsort(stats[:, -1],)[-2:-20:-1]


# Plot the connected components
for component_id in top_components_index:
  condition_array = np.rollaxis(np.stack((labels == component_id, labels == component_id, labels == component_id)), 0, 3)
  component = np.where(condition_array, road_data[0], np.zeros_like(road_data[0]))
  contour, hierarchy = cv2.findContours(component[:,:,0], cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

  cv2.drawContours(component, contour, -1, (255,0,0),2)
  plt.matshow(component)
  #plt.title(f'size:{stats[component_id,-1]}')
plt.show()

In [0]:
contour, hierarchy = cv2.findContours(road_data[0][:,:,0], cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
img = np.copy(road_data[0])
cv2.drawContours(img, contour, -1, (255,0,0),2)
plt.imshow(img)


In [0]:
list(range(-1,-12,-1))

In [0]:
tmp = np.sqrt(np.square(grad_x_scaled))
tmp = (tmp - tmp.min())/(tmp.max() - tmp.min())

In [0]:
plt.imshow(bin_grad_combined[1], cmap='gray')

In [0]:
plt.imshow(bin_grad_combined, cmap='gray');

In [0]:
# Find connected components
retval, labels, stats, centroids = cv2.connectedComponentsWithStats(bin_grad_x[1])

In [0]:
# Find the top 10 components
top_components_index = np.argsort(stats[:, -1])[-30:-1]
top_components = stats[top_components_index, -1]

In [0]:
for component_id in top_components_index:
  condition_array = np.rollaxis(np.stack((labels == component_id, labels == component_id, labels == component_id)), 0, 3)
  component = np.where(condition_array, road_data[0], np.zeros_like(road_data[0]))
  plt.imshow(component)
  plt.show()

In [0]:
test = (grad_x**2) + (grad_y**2)

plt.imshow(np.sqrt(test))

In [0]:
from google.colab import drive
drive.mount('/content/drive')